<a href="https://colab.research.google.com/github/vinods03/deep_learning/blob/main/NLP_9b_Pyspark_Project_Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 43.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f8ec4799d1cc281b66b8da7e1abb0b3bd1a47061650e9700d1a37a91891aea52
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Spam Detection').getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = spark.read.csv('/content/drive/MyDrive/smsspamcollection/SMSSpamCollection',inferSchema = True, sep = '\t')

In [ ]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [ ]:
data.describe()

DataFrame[summary: string, _c0: string, _c1: string]

In [ ]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [ ]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import length

In [ ]:
data = data.withColumn('length',length(data['text']))

In [ ]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [ ]:
# see if length of text is different for ham and spam. Yes, it is !!

data.groupBy('class').mean('length').show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [ ]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, HashingTF, IDF, CountVectorizer, StringIndexer, VectorAssembler

In [ ]:
tokenizer = Tokenizer(inputCol = 'text', outputCol = 'words')

In [ ]:
remover = StopWordsRemover(inputCol = 'words', outputCol = 'required_words')

In [ ]:
vectorizer = CountVectorizer(inputCol = 'required_words', outputCol = 'c_vec')

In [ ]:
# hashing_TF = HashingTF(inputCol = 'required_words', outputCol = 'rawFeatures')

In [ ]:
idf = IDF(inputCol = 'c_vec', outputCol = 'tf_idf')

In [ ]:
assembler = VectorAssembler(inputCols = ['tf_idf', 'length'], outputCol = 'features')

In [ ]:
ham_spam_convert_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'label')

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages = [ham_spam_convert_to_numeric, tokenizer, remover, vectorizer, idf, assembler])

In [ ]:
fit_data_into_pipeline = pipeline.fit(data)

In [ ]:
transformed_data = fit_data_into_pipeline.transform(data)

In [ ]:
transformed_data.select('features', 'label').show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(13424,[7,11,31,6...|  0.0|
|(13424,[0,24,297,...|  0.0|
|(13424,[2,13,19,3...|  1.0|
|(13424,[0,70,80,1...|  0.0|
|(13424,[36,134,31...|  0.0|
|(13424,[10,60,139...|  1.0|
|(13424,[10,53,103...|  0.0|
|(13424,[125,184,4...|  0.0|
|(13424,[1,47,118,...|  1.0|
|(13424,[0,1,13,27...|  1.0|
|(13424,[18,43,120...|  0.0|
|(13424,[8,17,37,8...|  1.0|
|(13424,[13,30,47,...|  1.0|
|(13424,[39,96,217...|  0.0|
|(13424,[552,1697,...|  0.0|
|(13424,[30,109,11...|  1.0|
|(13424,[82,214,47...|  0.0|
|(13424,[0,2,49,13...|  0.0|
|(13424,[0,74,105,...|  0.0|
|(13424,[4,30,33,5...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [ ]:
transformed_data.printSchema

<bound method DataFrame.printSchema of DataFrame[class: string, text: string, length: int, label: double, words: array<string>, required_words: array<string>, c_vec: vector, tf_idf: vector, features: vector]>

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
nb = NaiveBayes()

In [ ]:
training, test = transformed_data.randomSplit([0.7, 0.3])

In [ ]:
spam_detector = nb.fit(training)

In [ ]:
predictions = spam_detector.transform(test)

In [ ]:
predictions.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|class|                text|length|label|               words|      required_words|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  ham| &lt;#&gt;  in mc...|    36|  0.0|[, &lt;#&gt;, , i...|[, &lt;#&gt;, , m...|(13423,[3,6,5140,...|(13423,[3,6,5140,...|(13424,[3,6,5140,...|[-278.45741315822...|[1.0,2.0921043904...|       0.0|
|  ham| said kiss, kiss,...|   133|  0.0|[, said, kiss,, k...|[, said, kiss,, k...|(13423,[3,94,212,...|(13423,[3,94,212,...|(13424,[3,94,212,...|[-1059.8528509708...|[1.0,2.6939016631...|       0.0|


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
eval = MulticlassClassificationEvaluator()

In [ ]:
accuracy = eval.evaluate(predictions)

In [ ]:
print(accuracy)

0.9179182753197234
